In [3]:
import pandas as pd
import numpy as np
from statsmodels.tsa.holtwinters import ExponentialSmoothing
import matplotlib.pyplot as plt
from datetime import timedelta
import pandas as pd
import io

In [4]:
file_path = "cafethai_prices.csv"
df = pd.read_csv(file_path)
df.head()

,date,time,cash_type,coffee_name,money
0,1/3/2023,08:04:19,card,Latte,75
1,1/3/2023,08:08:54,cash,Americano with Milk,70
2,1/3/2023,08:09:03,card,Cappuccino,75
3,1/3/2023,08:10:26,cash,Cappuccino,75
4,1/3/2023,08:15:37,cash,Cappuccino,75


In [5]:
#แปลงคอลัมน์'date'ให้เป็นรูปแบบวันที่ที่ถูกต้อง
df['date'] = pd.to_datetime(df['date'], format='%d/%m/%Y')

In [6]:
#สร้างคอลัมน์ใหม่สำหรับวันที่
df['day'] = df['date'].dt.date

In [7]:
#นับจำนวนแก้วที่ขายได้ในแต่ละวันสำหรับแต่ละเมนู
daily_sales = df.groupby(['day', 'coffee_name']).size().reset_index(name='cups_sold')

In [8]:
#คำนวณยอดขายเฉลี่ยรายวันสำหรับแต่ละเมนู
average_daily_sales = daily_sales.groupby('coffee_name')['cups_sold'].mean().reset_index()

In [9]:
#เปลี่ยนชื่อคอลัมน์เพื่อความชัดเจน
average_daily_sales.rename(columns={'cups_sold': 'average_cups_per_day'}, inplace=True)

In [10]:
#กำหนดจำนวนวันที่ต้องการทำนาย (60 วัน)
forecast_days = 60

In [11]:
#คาดการณ์ยอดขายรวมใน 60 วันข้างหน้า
average_daily_sales['forecasted_cups_60_days'] = average_daily_sales['average_cups_per_day'] * forecast_days

In [12]:
#แสดงผลการคาดการณ์ยอดขายต่อวัน
print("การคาดการณ์ยอดขายต่อวันสำหรับแต่ละเมนู:")
print(average_daily_sales[['coffee_name', 'average_cups_per_day']])
print("-" * 50)

การคาดการณ์ยอดขายต่อวันสำหรับแต่ละเมนู:
           coffee_name  average_cups_per_day
0            Americano             13.698734
1  Americano with Milk             19.158028
2           Cappuccino             12.381378
3                Cocoa              5.854139
4              Cortado              7.020699
5             Espresso              3.698592
6        Hot Chocolate              6.977951
7                Latte             18.695322
--------------------------------------------------


In [13]:
#แสดงผลการคาดการณ์ยอดขายรวมใน 60 วันข้างหน้า
print("การคาดการณ์ยอดขายรวมใน 60 วันข้างหน้าสำหรับแต่ละเมนู:")
print(average_daily_sales[['coffee_name', 'forecasted_cups_60_days']])
print("-" * 50)

การคาดการณ์ยอดขายรวมใน 60 วันข้างหน้าสำหรับแต่ละเมนู:
           coffee_name  forecasted_cups_60_days
0            Americano               821.924051
1  Americano with Milk              1149.481669
2           Cappuccino               742.882653
3                Cocoa               351.248357
4              Cortado               421.241915
5             Espresso               221.915493
6        Hot Chocolate               418.677043
7                Latte              1121.719343
--------------------------------------------------


In [14]:
# สร้างกราฟแท่ง
plt.bar(average_daily_sales['coffee_name'], average_daily_sales['forecasted_cups_60_days'], color='skyblue')

# เพิ่มรายละเอียดบนกราฟ
plt.title('Total sales in the next 60 days menu', fontsize=16)
plt.xlabel('Menu', fontsize=12)
plt.ylabel('Total number of glasses expected to be sold (in 60 days))', fontsize=12)
plt.xticks(rotation=45, ha='right') # หมุนชื่อแกน X เพื่อให้อ่านง่าย
plt.grid(axis='y', linestyle='--', alpha=0.7) # เพิ่มเส้นกริดแนวนอน

# แสดงค่าบนแต่ละแท่งกราฟ
for index, value in enumerate(average_daily_sales['forecasted_cups_60_days']):
    plt.text(index, value + 0.5, f'{value:.0f}', ha='center', va='bottom')

# แสดงกราฟ
plt.show()